In [2]:
import pandas as pd
import numpy as np
import time

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules

In [3]:
# read dataframe
Rawdata = pd.read_csv('./deal.csv', header = 0)
Rawdata.head(10)

,INVOICE_NO,CUST_ID,ITEM_ID,ITEM_NO,PRODUCT_TYPE,TRX_DATE,QUANTITY
0,CX47348203,3218,3217532,M25P40-VMN6TPB,MEMORY_EMBEDED,2016/7/26,2500
1,CX47346522,2470,3326781,AU80610006237AASLBX9,CPU / MPU,2016/7/11,50
2,CX47348534,16135,740487,MMBD2837LT1G,DISCRETE,2016/7/27,3000
3,A20160700174,999999999,3434776,IHLP1616ABER2R2M11,PEMCO,2016/7/29,0
4,CX47346184,2356,70072,MMBT3906LT1G,DISCRETE,2016/7/6,12000
5,A10085337,2506,3204503,PCA9555DWR,LOGIC IC,2016/7/21,0
6,CX47347899,10228,3420352,TMP103AYFFR,LINEAR IC,2016/7/25,3000
7,CX47346191,38381,3311565,OV6922-V09N,OPTICAL AND SENSOR,2016/7/6,1152
8,5119,999999999,140887,SN74AHC1G126DCKR,LOGIC IC,2016/7/31,0
9,CX47346636,27495,3216410,SI2303CDS-T1-GE3,DISCRETE,2016/7/11,3000


In [4]:
print("Before: ", Rawdata.shape)
df = Rawdata

# delete the row if QUANTITY<=0
df = df[df.QUANTITY > 0]

# delete Nan or NULL
df.dropna()

# delete duplicates
df.drop_duplicates(inplace=True)

print("After: ", df.shape)
df.info()
df.head(10)

Before:  (157396, 7)
After:  (111961, 7)
<class 'pandas.core.frame.DataFrame'>
Index: 111961 entries, 0 to 157395
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   INVOICE_NO    111961 non-null  object
 1   CUST_ID       111961 non-null  int64 
 2   ITEM_ID       111961 non-null  int64 
 3   ITEM_NO       111961 non-null  object
 4   PRODUCT_TYPE  111961 non-null  object
 5   TRX_DATE      111961 non-null  object
 6   QUANTITY      111961 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 6.8+ MB


C:\Users\MB207\AppData\Local\Temp\ipykernel_22036\89577378.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop_duplicates(inplace=True)


,INVOICE_NO,CUST_ID,ITEM_ID,ITEM_NO,PRODUCT_TYPE,TRX_DATE,QUANTITY
0,CX47348203,3218,3217532,M25P40-VMN6TPB,MEMORY_EMBEDED,2016/7/26,2500
1,CX47346522,2470,3326781,AU80610006237AASLBX9,CPU / MPU,2016/7/11,50
2,CX47348534,16135,740487,MMBD2837LT1G,DISCRETE,2016/7/27,3000
4,CX47346184,2356,70072,MMBT3906LT1G,DISCRETE,2016/7/6,12000
6,CX47347899,10228,3420352,TMP103AYFFR,LINEAR IC,2016/7/25,3000
7,CX47346191,38381,3311565,OV6922-V09N,OPTICAL AND SENSOR,2016/7/6,1152
9,CX47346636,27495,3216410,SI2303CDS-T1-GE3,DISCRETE,2016/7/11,3000
10,CX47348680,30377,123164,TLZ24B-GS08,DISCRETE,2016/7/28,2500
12,CX47345496,16686,14831058,QCA9550-AT4A,CHIPSET / ASP,2016/7/1,651
13,CX47348656,2506,148734,SN74LVC1G04DBVR,LOGIC IC,2016/7/28,3000


In [5]:
invoice = pd.unique(df['INVOICE_NO'])
orderList = []

for i in invoice:
    number = df[df['INVOICE_NO'] == i]
    product = number['PRODUCT_TYPE']
    product = pd.unique(product)
    orderList.append(product)

print("invoice:", len(invoice),  "  orderList:", len(orderList))
orderList

invoice: 36498   orderList: 36498


[array(['MEMORY_EMBEDED'], dtype=object),
 array(['CPU / MPU'], dtype=object),
 array(['DISCRETE'], dtype=object),
 array(['DISCRETE'], dtype=object),
 array(['LINEAR IC', 'LOGIC IC', 'OTHERS'], dtype=object),
 array(['OPTICAL AND SENSOR'], dtype=object),
 array(['DISCRETE'], dtype=object),
 array(['DISCRETE'], dtype=object),
 array(['CHIPSET / ASP'], dtype=object),
 array(['LOGIC IC', 'LINEAR IC', 'DISCRETE', 'OPTICAL AND SENSOR'],
       dtype=object),
 array(['LOGIC IC', 'LINEAR IC'], dtype=object),
 array(['LINEAR IC'], dtype=object),
 array(['MEMORY_SYSTEM', 'LINEAR IC', 'MEMORY_EMBEDED'], dtype=object),
 array(['CPU / MPU'], dtype=object),
 array(['PEMCO'], dtype=object),
 array(['LINEAR IC'], dtype=object),
 array(['DISCRETE'], dtype=object),
 array(['DISCRETE'], dtype=object),
 array(['DISCRETE'], dtype=object),
 array(['PEMCO', 'DISCRETE'], dtype=object),
 array(['DISCRETE', 'LINEAR IC'], dtype=object),
 array(['LOGIC IC'], dtype=object),
 array(['CPU / MPU'], dtype=object),
 

In [6]:
# 轉換成模型所需的input形式
te = TransactionEncoder()  # 定义模型
ordreList_tf = te.fit_transform(orderList)
ordreList_tf = pd.DataFrame(ordreList_tf, columns=te.columns_)


# apriori
random_support = np.arange(0.001,0.009,0.001)  
random_confident = np.arange(0.01,0.015,0.002)
print("【apriori】")

for sup in random_support:
    for conf in random_confident: 
        start = time.time()
        apriori_frequent_itemsets = apriori(ordreList_tf, min_support = sup, use_colnames = True) # 求频繁项集        
        apriori_association_rule = association_rules(apriori_frequent_itemsets, metric = 'confidence', min_threshold = conf) # 求关联规则
        end = time.time()        
        print("support: ", sup, "confident: ", conf, "execute time: ", end-start, "counts: ", len(apriori_association_rule))        

【apriori】
support:  0.001 confident:  0.01 execute time:  0.052004337310791016 counts:  201
support:  0.001 confident:  0.012 execute time:  0.04200458526611328 counts:  190
support:  0.001 confident:  0.014 execute time:  0.039000511169433594 counts:  182
support:  0.002 confident:  0.01 execute time:  0.032004356384277344 counts:  135
support:  0.002 confident:  0.012 execute time:  0.044002532958984375 counts:  130
support:  0.002 confident:  0.014 execute time:  0.04100227355957031 counts:  124
support:  0.003 confident:  0.01 execute time:  0.033000946044921875 counts:  72
support:  0.003 confident:  0.012 execute time:  0.027001380920410156 counts:  71
support:  0.003 confident:  0.014 execute time:  0.02300286293029785 counts:  65
support:  0.004 confident:  0.01 execute time:  0.02200174331665039 counts:  36
support:  0.004 confident:  0.012 execute time:  0.021000385284423828 counts:  36
support:  0.004 confident:  0.014 execute time:  0.023119688034057617 counts:  36
support:

In [7]:
# FP-Growth

random_support = np.arange(0.001,0.009,0.001)  
random_confident = np.arange(0.01,0.015,0.002)
print("【FP-Growth】")

for sup in random_support:
    for conf in random_confident: 
        start = time.time()
        fpgrowths_frequent_itemsets = apriori(ordreList_tf, min_support = sup, use_colnames = True) # 求频繁项集        
        fpgrowths_association_rule = association_rules(fpgrowths_frequent_itemsets, metric = 'confidence', min_threshold = conf) # 求关联规则
        end = time.time()
        print("support: ", sup, "confident: ", conf, "execute time: ", end-start, "counts: ", len(fpgrowths_association_rule))        

【FP-Growth】
support:  0.001 confident:  0.01 execute time:  0.06600689888000488 counts:  201
support:  0.001 confident:  0.012 execute time:  0.05000162124633789 counts:  190
support:  0.001 confident:  0.014 execute time:  0.05800294876098633 counts:  182
support:  0.002 confident:  0.01 execute time:  0.03600192070007324 counts:  135
support:  0.002 confident:  0.012 execute time:  0.04031991958618164 counts:  130
support:  0.002 confident:  0.014 execute time:  0.046001434326171875 counts:  124
support:  0.003 confident:  0.01 execute time:  0.034001827239990234 counts:  72
support:  0.003 confident:  0.012 execute time:  0.03700542449951172 counts:  71
support:  0.003 confident:  0.014 execute time:  0.04400062561035156 counts:  65
support:  0.004 confident:  0.01 execute time:  0.027000904083251953 counts:  36
support:  0.004 confident:  0.012 execute time:  0.02600407600402832 counts:  36
support:  0.004 confident:  0.014 execute time:  0.029803991317749023 counts:  36
support:  

In [12]:
fpgrowths_frequent_itemsets

,support,itemsets
0,0.044112,(CHIPSET / ASP)
1,0.110746,(CPU / MPU)
2,0.257905,(DISCRETE)
3,0.260398,(LINEAR IC)
4,0.121541,(LOGIC IC)
5,0.066716,(MEMORY_EMBEDED)
6,0.021234,(MEMORY_SYSTEM)
7,0.039865,(OPTICAL AND SENSOR)
8,0.113403,(OTHERS)
9,0.119486,(PEMCO)


### 關聯規則輸出csv ###

In [15]:
frequent_itemset = apriori(ordreList_tf, min_support = 0.004, use_colnames = True) # 求频繁项集
association_rule = association_rules(frequent_itemset, metric = 'confidence', min_threshold=0.014)
association_rule.to_csv('E:/Sandy/COURSE/DM/HW4/apriori_association_rule.csv')

### 商品推薦功能 ###

In [28]:
import re
#做資料處理
def str(x):
    return re.sub("frozenset|\'|\\{|\\}|\\(|\\)","",x)

associationDF = pd.read_csv("E:/Sandy/COURSE/DM/HW4/apriori_association_rule.csv")
associationDF = associationDF.drop(['Unnamed: 0'],axis=1)
associationDF['antecedents'] = associationDF['antecedents'].apply(lambda x: str(x))
associationDF['consequents'] = associationDF['consequents'].apply(lambda x: str(x))

associationDF

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,CHIPSET / ASP,CPU / MPU,0.044112,0.110746,0.005343,0.121118,1.093658,0.000458,1.011802,0.089589
1,CPU / MPU,CHIPSET / ASP,0.110746,0.044112,0.005343,0.048243,1.093658,0.000458,1.004341,0.096302
2,CPU / MPU,LINEAR IC,0.110746,0.260398,0.007672,0.069273,0.266026,-0.021166,0.794650,-0.756254
3,LINEAR IC,CPU / MPU,0.260398,0.110746,0.007672,0.029461,0.266026,-0.021166,0.916248,-0.788602
4,CPU / MPU,LOGIC IC,0.110746,0.121541,0.005206,0.047006,0.386754,-0.008254,0.921789,-0.640688
5,LOGIC IC,CPU / MPU,0.121541,0.110746,0.005206,0.042831,0.386754,-0.008254,0.929047,-0.643494
6,OTHERS,CPU / MPU,0.113403,0.110746,0.005288,0.046630,0.421051,-0.007271,0.932748,-0.607979
7,CPU / MPU,OTHERS,0.110746,0.113403,0.005288,0.047749,0.421051,-0.007271,0.931053,-0.607266
8,DISCRETE,LINEAR IC,0.257905,0.260398,0.034906,0.135345,0.519761,-0.032252,0.855372,-0.554579
9,LINEAR IC,DISCRETE,0.260398,0.257905,0.034906,0.134049,0.519761,-0.032252,0.856971,-0.555411


In [29]:
associationDF['antecedents']

0           CHIPSET / ASP
1               CPU / MPU
2               CPU / MPU
3               LINEAR IC
4               CPU / MPU
5                LOGIC IC
6                  OTHERS
7               CPU / MPU
8                DISCRETE
9               LINEAR IC
10               DISCRETE
11               LOGIC IC
12         MEMORY_EMBEDED
13               DISCRETE
14               DISCRETE
15     OPTICAL AND SENSOR
16                 OTHERS
17               DISCRETE
18                  PEMCO
19               DISCRETE
20              LINEAR IC
21               LOGIC IC
22         MEMORY_EMBEDED
23              LINEAR IC
24                 OTHERS
25              LINEAR IC
26         MEMORY_EMBEDED
27               LOGIC IC
28                 OTHERS
29          MEMORY_SYSTEM
30    DISCRETE, LINEAR IC
31     DISCRETE, LOGIC IC
32    LINEAR IC, LOGIC IC
33               DISCRETE
34              LINEAR IC
35               LOGIC IC
Name: antecedents, dtype: object

In [35]:
product = input("請輸入你想要的產品(若多項請以, 隔開) : ")#多個範例(LINEAR IC, DISCRETE)
print('你選擇的產品:',product)

product=product.replace(' ','').split(',')
product=set(product)

# product = ",".join(product)
#print(product)


for count,i in enumerate(associationDF['antecedents']):
    x=i.replace(' ','').split(',')
    x=set(x)
    if(x==product):
        result=associationDF[associationDF['antecedents']==i]['consequents']
        result= ",".join(result).split(',')
        result=pd.unique([a.strip() for a in result])
        print('推薦商品:',result)
        break
    if(count==len(associationDF['antecedents'])-1):
        print('找不到與您選擇有相關的產品！')


        
    

# associate_set=df_association[set(df_association['antecedents'])==set(product)]['consequents']
# associate_set= ",".join(associate_set)
# #associate_set=associate_set.replace(' ','')
# associate_set=pd.unique(associate_set.split(','))

# associate_set=" , ".join(associate_set)
# if(associate_set==""):
#     print('找不到與您選擇有相關的產品！')
# else:    
#     print('你可能也有興趣的東西:',associate_set)
         





你選擇的產品: LINEAR IC, LOGIC IC
推薦商品: ['DISCRETE']


C:\Users\MB207\AppData\Local\Temp\ipykernel_22036\1528202353.py:17: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  result=pd.unique([a.strip() for a in result])
